In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import re

In [2]:
driver = webdriver.Chrome(executable_path=r"C:\Users\Mark\Anaconda3\selenium\webdriver\chrome\chromedriver.exe")

In [3]:
driver.get('https://www.beeradvocate.com/lists/popular/')
beer_html = driver.page_source
beer_soup = BeautifulSoup(beer_html, 'lxml')

In [4]:
# name and url
beer_list = []
for i in beer_soup.find_all('td', {'valign': 'middle'})[4:]:
    beer_dict = {}
    beer_dict['name'] = i.find('a').text
    beer_dict['url'] = i.find('a').attrs['href']
    beer_list.append(beer_dict)

df_beer = pd.DataFrame(beer_list)
df_beer.index += 1

# score
beer_score = []
for i in beer_soup.find_all('td', {'class': 'hr_bottom_light', 'align':'left', 'valign':'top'}):
    beer_score.append(i.text)
df_beer['score'] = beer_score

# ratings count
beer_ratings = []
for i in beer_soup.find_all('td', {'class': 'hr_bottom_light', 'align':'right'}):
    beer_ratings.append(i.text)
df_beer['ratings'] = beer_ratings

In [5]:
df_beer.head()

,name,url,score,ratings
1,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948"
2,90 Minute IPA,/beer/profile/64/2093/,4.27,"15,783"
3,Pliny The Elder,/beer/profile/863/7971/,4.65,"14,754"
4,Two Hearted Ale,/beer/profile/287/1093/,4.27,"14,621"
5,Sculpin IPA,/beer/profile/199/29619/,4.39,"14,146"


In [6]:
abv_grabber = lambda x: re.search('\(ABV\): (\d+\.\d+)%', x).group(1)
notes_grabber = lambda x: re.search('Description:[\s\S]+ IBU', x).group(0)
ibu_grabber = lambda x: re.search('(\d*) IBU', x).group(1)
review_grabber = lambda x: re.search('overall:\s\d\.\d*(.*[\s\S]*\d characters)', x).group(1)

def notes_cleaner(string):
    x = notes_grabber(string)
    x = re.sub('\s', ' ', x)
    x = re.sub('\d* IBU', '', x)
    x = re.sub('Availability:', '', x)
    x = re.sub('Description:', '', x)
    x = re.sub(',', '', x)
    return x

def review_cleaner(string):
    x = review_grabber(string)
    x = re.sub('\d* characters', '', x)
    x = re.sub('\n', ' ', x)
    x = re.sub(',', '', x)
    return x

In [118]:
beer_url = []

brewery_list = []
abv_list = []
notes_list = []

user_score = []
score_breakdown = []
user_names = []
user_review = []

for url in df_beer['url'][:2]:
    driver.get(f'https://www.beeradvocate.com{url}')
    sleep(2)
    beer_page = driver.page_source
    beer_page_soup = BeautifulSoup(beer_page, 'lxml')
    
#     get the brewery name
    for i in beer_page_soup.find_all('div', {'id':'info_box'}):
        brewery_list.append(i.find('a').text)
    
    print(brewery_list[-1])

#     get the notes and abv
    beer_text = beer_page_soup.find('div', {'id':'info_box'}).text
    abv_list.append(abv_grabber(beer_text))
    notes_list.append (notes_cleaner(notes_grabber(beer_text)))
    
    print('    abv/notes done')
    
#     get user_score, score breakdwon, user names, add url in order to map back to df_beers
    for i in beer_page_soup.find_all('div', {'id':'rating_fullview_content_2'}):
        user_score.append(i.find('span', {'class': 'BAscore_norm'}).text)
        score_breakdown.append(i.find('span', {'class': 'muted'}).text)
        user_names.append(i.find('a', {'class':'username'}).text)
        
        beer_url.append(url)
        
    print('    score, breakdown, name, url done')
    
#     get the user reviews
    for i in range(1,26):
        try:
            review = review_grabber(beer_page_soup.find_all('div', {'id':'rating_fullview_content_2'})[i].text)
            user_review.append(re.sub('\d* characters', '', review))
        except:
            user_review.append("No Review")
    
    print('    user reivews done')
    
    
    
    sleep(2)

Founders Brewing Company
    abv/notes done
    score, breakdown, name, url done
    user reivews done
Dogfish Head Craft Brewery
    abv/notes done
    score, breakdown, name, url done
    user reivews done


In [126]:
beer_url[:26]

['/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/64/2093/']

In [120]:
abv_list

['8.30', '9.00']

In [121]:
notes_list

['     Sep - Dec  Brewed with flaked oats, bitter and sweetened imported chocolates, Sumatra and Kona coffee.  ',
 '    Esquire Magazine calls our 90 Minute IPA "perhaps the best IPA in America." An imperial IPA best savored from a snifter, 90 Minute has a great malt backbone that stands up to the extreme hopping rate.  90 Minute IPA was the first beer we continuously hopped, allowing for a pungent -- but not crushing -- hop flavor.  ']

In [127]:
len(user_score)

50

In [123]:
score_breakdown[:5]

['look: 4.25 | smell: 4.75 | taste: 5 | feel: 4.75 |  overall: 4.75',
 'look: 5 | smell: 4.75 | taste: 4.75 | feel: 5 |  overall: 4.75',
 'look: 4 | smell: 4 | taste: 4 | feel: 4 |  overall: 4',
 'look: 4 | smell: 4.25 | taste: 4 | feel: 4.25 |  overall: 4.25',
 'look: 4.25 | smell: 4 | taste: 4 | feel: 4.25 |  overall: 4.25']

In [124]:
user_names[:5]

['Nobel96', 'Batman11', 'Bloodsnot77', 'jrc1093', 'Maudrey']

In [125]:
user_review[:5]

['No Review',
 'No Review',
 'No Review',
 'No Review',
 '5Coffee forward beer that is good but not great. And I’m used to greatness from founders when it comes to their stouts.']

In [21]:
# static function:

brewery_list = []
abv_list = []
notes_list = []
ibu_list = []


def get_reviews_static_data(url):
    driver.get(f'https://www.beeradvocate.com{url}')
    sleep(2)
    beer_page = driver.page_source
    beer_page_soup = BeautifulSoup(beer_page, 'lxml')
    
#     get the brewery name
    for i in beer_page_soup.find_all('div', {'id':'info_box'}):
        brewery_list.append(i.find('a').text)
    
    
    print(brewery_list[-1])
    print(f'    https://www.beeradvocate.com{url}')

#     get the notes and abv
    beer_text = beer_page_soup.find('div', {'id':'info_box'}).text
    abv_list.append(abv_grabber(beer_text))
    try:
        notes_list.append(notes_cleaner(notes_grabber(beer_text)))
        ibu_list.append(ibu_grabber(beer_text))
    except:
        notes_list.append('No Notes')
        ibu_list.append('No IBU')
    
    print('    abv/notes/ibu done')
    
    sleep(2)

In [22]:
df_beer['url'].apply(get_reviews_static_data)

Founders Brewing Company
    https://www.beeradvocate.com/beer/profile/1199/11757/
    abv/notes/ibu done
Dogfish Head Craft Brewery
    https://www.beeradvocate.com/beer/profile/64/2093/
    abv/notes/ibu done
Russian River Brewing Company
    https://www.beeradvocate.com/beer/profile/863/7971/
    abv/notes/ibu done
Bell's Brewery - Eccentric Café & General Store
    https://www.beeradvocate.com/beer/profile/287/1093/
    abv/notes/ibu done
Ballast Point Brewing Company
    https://www.beeradvocate.com/beer/profile/199/29619/
    abv/notes/ibu done
The Alchemist Brewery and Visitors Center
    https://www.beeradvocate.com/beer/profile/46317/16814/
    abv/notes/ibu done
Bell's Brewery - Eccentric Café & General Store
    https://www.beeradvocate.com/beer/profile/287/17112/
    abv/notes/ibu done
Founders Brewing Company
    https://www.beeradvocate.com/beer/profile/1199/19960/
    abv/notes/ibu done
Goose Island Beer Co.
    https://www.beeradvocate.com/beer/profile/1146/10672/
    a

1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
30     None
       ... 
221    None
222    None
223    None
224    None
225    None
226    None
227    None
228    None
229    None
230    None
231    None
232    None
233    None
234    None
235    None
236    None
237    None
238    None
239    None
240    None
241    None
242    None
243    None
244    None
245    None
246    None
247    None
248    None
249    None
250    None
Name: url, Length: 250, dtype: object

In [23]:
brewery_list

['Founders Brewing Company',
 'Dogfish Head Craft Brewery',
 'Russian River Brewing Company',
 "Bell's Brewery - Eccentric Café & General Store",
 'Ballast Point Brewing Company',
 'The Alchemist Brewery and Visitors Center',
 "Bell's Brewery - Eccentric Café & General Store",
 'Founders Brewing Company',
 'Goose Island Beer Co.',
 'North Coast Brewing Co.',
 'Dogfish Head Craft Brewery',
 'Sierra Nevada Brewing Co.',
 'Stone Brewing',
 'Sierra Nevada Brewing Co.',
 'Stone Brewing',
 'Arrogant Brewing',
 'Sierra Nevada Brewing Co.',
 '3 Floyds Brewing Co.',
 'Lagunitas Brewing Company',
 'Unibroue',
 'Oskar Blues Grill & Brew',
 'Brouwerij St. Bernardus NV',
 'Brasserie de Rochefort',
 'Boston Beer Company (Samuel Adams)',
 'Lagunitas Brewing Company',
 'Lagunitas Brewing Company',
 'New Belgium Brewing',
 'Tröegs Brewing Company',
 'Bayerische Staatsbrauerei Weihenstephan',
 'Founders Brewing Company',
 'Founders Brewing Company',
 'Dogfish Head Craft Brewery',
 'Brouwerij Duvel Moort

In [24]:
abv_list

['8.30',
 '9.00',
 '8.00',
 '7.00',
 '7.00',
 '8.00',
 '10.00',
 '11.80',
 '14.10',
 '9.00',
 '6.00',
 '5.60',
 '6.90',
 '7.20',
 '9.40',
 '7.20',
 '6.80',
 '6.20',
 '7.85',
 '9.00',
 '10.50',
 '10.00',
 '11.30',
 '5.00',
 '7.50',
 '6.20',
 '5.20',
 '7.50',
 '5.40',
 '7.20',
 '11.20',
 '18.00',
 '8.50',
 '7.50',
 '8.10',
 '9.00',
 '4.20',
 '8.00',
 '7.50',
 '5.30',
 '8.50',
 '6.80',
 '8.60',
 '9.60',
 '6.50',
 '6.50',
 '6.70',
 '4.70',
 '11.40',
 '12.90',
 '5.40',
 '10.00',
 '12.00',
 '6.00',
 '5.80',
 '10.00',
 '7.50',
 '5.00',
 '11.00',
 '8.70',
 '14.00',
 '7.00',
 '10.00',
 '6.70',
 '6.66',
 '4.50',
 '6.00',
 '9.70',
 '10.20',
 '7.00',
 '9.40',
 '5.00',
 '9.60',
 '10.50',
 '5.20',
 '13.00',
 '7.20',
 '7.00',
 '9.50',
 '7.70',
 '5.60',
 '4.90',
 '4.20',
 '11.00',
 '9.10',
 '6.50',
 '8.70',
 '5.60',
 '12.00',
 '8.50',
 '10.80',
 '6.10',
 '5.20',
 '10.00',
 '5.10',
 '9.50',
 '4.74',
 '10.20',
 '6.60',
 '5.30',
 '8.20',
 '9.50',
 '5.00',
 '6.20',
 '5.90',
 '5.10',
 '10.00',
 '11.70',
 '

In [25]:
notes_list

['     Sep - Dec  Brewed with flaked oats bitter and sweetened imported chocolates Sumatra and Kona coffee.  ',
 '    Esquire Magazine calls our 90 Minute IPA "perhaps the best IPA in America." An imperial IPA best savored from a snifter 90 Minute has a great malt backbone that stands up to the extreme hopping rate.  90 Minute IPA was the first beer we continuously hopped allowing for a pungent -- but not crushing -- hop flavor.  ',
 'No Notes',
 'No Notes',
 'No Notes',
 '    "An American Double IPA" ',
 'No Notes',
 'No Notes',
 '    ',
 'No Notes',
 "    60 Minute IPA is continuously hopped -- more than 60 hop additions over a 60-minute boil. (Getting a vibe of where the name came from?)  60 Minute is brewed with a slew of great Northwest hops. A powerful but balanced East Coast IPA with a lot of citrusy hop character it's the session beer for hardcore enthusiasts!  ",
 '    Generous quantities of premium Cascade hops give the Pale Ale its fragrant bouquet and spicy flavor.  ',
 'No

In [26]:
ibu_list

['60',
 '90',
 'No IBU',
 'No IBU',
 'No IBU',
 '75',
 'No IBU',
 'No IBU',
 '60',
 'No IBU',
 '60',
 '37',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 '60',
 'No IBU',
 'No IBU',
 '98',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 '6',
 'No IBU',
 '93',
 'No IBU',
 '65',
 'No IBU',
 '120',
 'No IBU',
 '70',
 '95',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 '50',
 'No IBU',
 'No IBU',
 'No IBU',
 '45',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 '28',
 '70',
 'No IBU',
 '68',
 'No IBU',
 '37',
 'No IBU',
 'No IBU',
 'No IBU',
 '86',
 'No IBU',
 '100',
 'No IBU',
 'No IBU',
 'No IBU',
 '50',
 'No IBU',
 '100',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 '70',
 'No IBU',
 '35',
 '50',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 'No IBU',
 '20',
 'No IBU',
 'No IBU',
 '102',
 'No IBU',
 'No IBU',
 'No IBU',
 '

In [27]:
df_beer['brewery'] = brewery_list
df_beer['abv'] = abv_list
df_beer['ibu'] = ibu_list
df_beer['brewer_notes'] = notes_list
df_beer.head()

,name,url,score,ratings,brewery,abv,ibu,brewer_notes
1,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,8.30,60,Sep - Dec Brewed with flaked oats bitter...
2,90 Minute IPA,/beer/profile/64/2093/,4.27,"15,783",Dogfish Head Craft Brewery,9.00,90,"Esquire Magazine calls our 90 Minute IPA ""..."
3,Pliny The Elder,/beer/profile/863/7971/,4.65,"14,754",Russian River Brewing Company,8.00,No IBU,No Notes
4,Two Hearted Ale,/beer/profile/287/1093/,4.27,"14,621",Bell's Brewery - Eccentric Café & General Store,7.00,No IBU,No Notes
5,Sculpin IPA,/beer/profile/199/29619/,4.39,"14,146",Ballast Point Brewing Company,7.00,No IBU,No Notes


In [28]:
# df_beer.to_csv('./df_beer.csv', index=False)

In [214]:
counter = 0
for i in range(0,3):
    print(i)
    print('hello')
    counter += 1
    print(counter)

0
hello
1
1
hello
2
2
hello
3


In [192]:
df_beer.head()

,name,url,score,ratings,Brewery,ABV,IBU
1,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,8.30,60
2,90 Minute IPA,/beer/profile/64/2093/,4.27,"15,782",Dogfish Head Craft Brewery,9.00,90
3,Pliny The Elder,/beer/profile/863/7971/,4.65,"14,754",Russian River Brewing Company,8.00,No IBU
4,Two Hearted Ale,/beer/profile/287/1093/,4.27,"14,619",Bell's Brewery - Eccentric Café & General Store,7.00,No IBU
5,Sculpin IPA,/beer/profile/199/29619/,4.39,"14,146",Ballast Point Brewing Company,7.00,No IBU


### Clicking through multiple pages

In [246]:
counter = 0
beer_url = []

user_score = []
score_breakdown = []
user_names = []
user_review = []

for url in df_beer['url'][:1]:
    for i in range(0,2):
        driver.get(f'https://www.beeradvocate.com{url}?view=beer&sort=&start={i*25}')
        sleep(2)
        beer_page = driver.page_source
        beer_page_soup = BeautifulSoup(beer_page, 'lxml')
        
        counter += 1
        print(f'page {counter}')
    
#     get user_score, score breakdwon, user names, add url in order to map back to df_beers
        for i in beer_page_soup.find_all('div', {'id':'rating_fullview_content_2'}):
            user_score.append(i.find('span', {'class': 'BAscore_norm'}).text)
            score_breakdown.append(i.find('span', {'class': 'muted'}).text)
            user_names.append(i.find('a', {'class':'username'}).text)

            beer_url.append(url)

        print('    score, breakdown, name, url done')
    
#     get the user reviews
        for i in range(1,26):
            try:
                review = review_cleaner(beer_page_soup.find_all('div', {'id':'rating_fullview_content_2'})[i].text)
                user_review.append(review)
            except:
                user_review.append("No Review")

        print('    user reivews done')
    
    
    
    sleep(2)

page 1
    score, breakdown, name, url done
    user reivews done
page 2
    score, breakdown, name, url done
    user reivews done


In [209]:
len(user_score)

50

In [210]:
score_breakdown

['look: 4.5 | smell: 4.25 | taste: 4.5 | feel: 4.25 |  overall: 4.25',
 'look: 4.5 | smell: 4.5 | taste: 4.25 | feel: 4.25 |  overall: 4.5',
 'look: 4.5 | smell: 4.5 | taste: 4.5 | feel: 4.5 |  overall: 4.5',
 'look: 4 | smell: 4 | taste: 3.5 | feel: 3.75 |  overall: 3.75',
 'look: 4.75 | smell: 4.5 | taste: 4.5 | feel: 1.75 |  overall: 4.5',
 'look: 4 | smell: 4 | taste: 4 | feel: 4 |  overall: 4',
 'look: 3.75 | smell: 4 | taste: 3.75 | feel: 4 |  overall: 4',
 'look: 4.25 | smell: 4 | taste: 4.5 | feel: 4.5 |  overall: 4.5',
 'look: 4 | smell: 4.75 | taste: 4.75 | feel: 4.5 |  overall: 4.5',
 'look: 3.75 | smell: 4 | taste: 4.5 | feel: 4 |  overall: 4.5',
 'look: 4.5 | smell: 4.5 | taste: 4.25 | feel: 4.25 |  overall: 4.25',
 'look: 5 | smell: 5 | taste: 5 | feel: 5 |  overall: 5',
 'look: 4 | smell: 4.25 | taste: 4.25 | feel: 4.5 |  overall: 4.25',
 'look: 4.25 | smell: 4.5 | taste: 3.75 | feel: 4.5 |  overall: 3.75',
 'look: 4 | smell: 4.25 | taste: 4.5 | feel: 4.5 |  overall: 4.2

In [231]:
user_names

['Nobel96',
 'Batman11',
 'Bloodsnot77',
 'jrc1093',
 'Maudrey',
 'ejmontoy1',
 'VforVinny',
 'lwillitz',
 'Bdasilva1077',
 'spongebob2',
 'OldIrishHermit',
 'Taterturnt',
 'Jpew31',
 'AWA',
 'Erin_Fitzpatrick',
 'Sjp2123',
 'Knickfan3',
 'bobstockwell',
 'smtilley',
 'First',
 'johnverduin',
 'MonkeyLord',
 'bmrattay',
 'Lars1265',
 'Sendji',
 'Almighty1',
 'TheAce',
 'Duey111',
 'Jesterr44',
 'dudiepants',
 'n2492004',
 'bctripp',
 'Podbay',
 'irishkyle21',
 'Fading2black',
 'SpaceBeagle',
 'Johng19751',
 'GTengineer',
 'Jakepeterson',
 'stevoj',
 'packerj',
 'TheBeerbarian',
 'zknight',
 'sunnovgoat',
 'dextr24',
 'pirateclem',
 'scream',
 'Dr_Gonzo',
 'Dirk_Chappelle',
 'Shy_Traveler45']

In [228]:
user_review

['No Review',
 'No Review',
 'No Review',
 'No Review',
 'Coffee forward beer that is good but not great. And I’m used to greatness from founders when it comes to their stouts.',
 'No Review',
 'No Review',
 'No Review',
 'No Review',
 'No Review',
 'An exceptional stout, the flavor is balanced, and the chocolate and coffee jump right out, with just the right amount mealyness in the background. The first half of the beer is a solid 5 in every category. The only reason I dropped the overall to a 4.25 is because the second half starts to become a bit overbearing in terms of sweetness and heavyness. Not a beer I could drink back to back but still an exceptional beer\xa0',
 'No Review',
 'No Review',
 'No Review',
 'No Review',
 'No Review',
 'No Review',
 'Its yummy and dark.  I like my beers yummy and dark and this damn beer is just that...yummy and dark',
 'No Review',
 'No Review',
 'A meal by its self!',
 'No Review',
 'No Review',
 'No Review',
 'No Review',
 'No Review',
 'No Review

In [ ]:
# making a function

In [254]:
beer_url = []

user_score = []
score_breakdown = []
user_names = []
user_review = []

# for url in df_beer['url'][:1]:
def get_user_reviews(url):
    counter = 0
    for i in range(0,2):
        driver.get(f'https://www.beeradvocate.com{url}?view=beer&sort=&start={i*25}')
        sleep(2)
        beer_page = driver.page_source
        beer_page_soup = BeautifulSoup(beer_page, 'lxml')
        
        counter += 1
        print(f'{url} -- page {counter}')
    
#     get user_score, score breakdwon, user names, add url in order to map back to df_beers
        for i in beer_page_soup.find_all('div', {'id':'rating_fullview_content_2'}):
            user_score.append(i.find('span', {'class': 'BAscore_norm'}).text)
            score_breakdown.append(i.find('span', {'class': 'muted'}).text)
            user_names.append(i.find('a', {'class':'username'}).text)

            beer_url.append(url)

        print('    score, breakdown, name, url done')
    
#     get the user reviews
        for i in range(1,26):
            try:
                review = review_cleaner(beer_page_soup.find_all('div', {'id':'rating_fullview_content_2'})[i].text)
                user_review.append(review)
            except:
                user_review.append("No Review")

        print('    user reivews done')
    
    
    
    sleep(2)

In [255]:
df_beer['url'][0:1].apply(get_user_reviews)

/beer/profile/1199/11757/ -- page 1
    score, breakdown, name, url done
    user reivews done
/beer/profile/1199/11757/ -- page 2
    score, breakdown, name, url done
    user reivews done
/beer/profile/64/2093/ -- page 1
    score, breakdown, name, url done
    user reivews done
/beer/profile/64/2093/ -- page 2
    score, breakdown, name, url done
    user reivews done


1    None
2    None
Name: url, dtype: object

In [256]:
beer_url

['/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/profile/1199/11757/',
 '/beer/pr

In [257]:
user_score

['4.82',
 '4.79',
 '4',
 '4.14',
 '4.09',
 '3.78',
 '4.08',
 '4.41',
 '4',
 '4.6',
 '4.74',
 '4.33',
 '3.05',
 '4.44',
 '4.21',
 '5',
 '4.29',
 '4.5',
 '4.7',
 '4.06',
 '4.37',
 '4.64',
 '3.87',
 '4.24',
 '2.1',
 '4.37',
 '4.38',
 '4.5',
 '3.73',
 '4.24',
 '4',
 '3.89',
 '4.37',
 '4.63',
 '4.29',
 '4.33',
 '5',
 '4.26',
 '4.04',
 '4.36',
 '4.75',
 '4.25',
 '4.14',
 '4.89',
 '4.91',
 '4.11',
 '3.83',
 '4.85',
 '4.58',
 '3.84',
 '4.29',
 '4',
 '4.01',
 '3.74',
 '4.44',
 '4.56',
 '4.33',
 '4.25',
 '4.64',
 '4.27',
 '4.33',
 '3.38',
 '4.57',
 '4.15',
 '3.98',
 '4.72',
 '4.14',
 '3.96',
 '4.24',
 '4.03',
 '4.1',
 '4.5',
 '4.27',
 '3.83',
 '5',
 '4',
 '3.6',
 '3.93',
 '4.18',
 '4.59',
 '4.65',
 '4.52',
 '4.17',
 '4.23',
 '4.09',
 '3.79',
 '4.23',
 '4.25',
 '4.11',
 '4.32',
 '4.25',
 '4.09',
 '4.65',
 '4.79',
 '4.01',
 '4.36',
 '4.31',
 '4.07',
 '4.09',
 '4.52']

In [258]:
score_breakdown

['look: 4.25 | smell: 4.75 | taste: 5 | feel: 4.75 |  overall: 4.75',
 'look: 5 | smell: 4.75 | taste: 4.75 | feel: 5 |  overall: 4.75',
 'look: 4 | smell: 4 | taste: 4 | feel: 4 |  overall: 4',
 'look: 4 | smell: 4.25 | taste: 4 | feel: 4.25 |  overall: 4.25',
 'look: 4.25 | smell: 4 | taste: 4 | feel: 4.25 |  overall: 4.25',
 'look: 4.5 | smell: 4.5 | taste: 3.5 | feel: 3.75 |  overall: 3.25',
 'look: 4.5 | smell: 4 | taste: 4 | feel: 4 |  overall: 4.25',
 'look: 4.75 | smell: 4.25 | taste: 4.5 | feel: 4 |  overall: 4.5',
 'look: 4 | smell: 4 | taste: 4 | feel: 4 |  overall: 4',
 'look: 4.5 | smell: 4.5 | taste: 4.75 | feel: 4.5 |  overall: 4.5',
 'look: 4.5 | smell: 4.75 | taste: 4.75 | feel: 4.75 |  overall: 4.75',
 'look: 4 | smell: 4.75 | taste: 4.25 | feel: 4 |  overall: 4.25',
 'look: 3 | smell: 3 | taste: 3 | feel: 3 |  overall: 3.25',
 'look: 4.5 | smell: 4.25 | taste: 4.5 | feel: 4.5 |  overall: 4.5',
 'look: 4.5 | smell: 4 | taste: 4.25 | feel: 4.25 |  overall: 4.25',
 'loo

In [259]:
user_names

['Nobel96',
 'Batman11',
 'Bloodsnot77',
 'jrc1093',
 'Maudrey',
 'ejmontoy1',
 'VforVinny',
 'lwillitz',
 'Bdasilva1077',
 'spongebob2',
 'OldIrishHermit',
 'Taterturnt',
 'Jpew31',
 'AWA',
 'Erin_Fitzpatrick',
 'Sjp2123',
 'Knickfan3',
 'bobstockwell',
 'smtilley',
 'First',
 'johnverduin',
 'MonkeyLord',
 'bmrattay',
 'Lars1265',
 'Sendji',
 'Almighty1',
 'TheAce',
 'Duey111',
 'Jesterr44',
 'dudiepants',
 'n2492004',
 'bctripp',
 'Podbay',
 'irishkyle21',
 'Fading2black',
 'SpaceBeagle',
 'Johng19751',
 'GTengineer',
 'Jakepeterson',
 'stevoj',
 'packerj',
 'TheBeerbarian',
 'zknight',
 'sunnovgoat',
 'dextr24',
 'pirateclem',
 'scream',
 'Dr_Gonzo',
 'Dirk_Chappelle',
 'Shy_Traveler45',
 'derdtheterd',
 'Jasoncpace',
 'Bridgtd',
 'BrentMcQuirk',
 'Bitburger',
 'MiniPigLover',
 'Afeo',
 'EMV',
 'RockySasquatch',
 'dudiepants',
 'MittenBreezeMD',
 'mrmdtighe',
 'DannyPaleAles',
 'zbelair',
 'SABERG',
 'Sable',
 'Shantto',
 'ToddSoonerFan',
 'Wapes',
 'BenHoppy',
 'Foxman27',
 'cmoor

In [260]:
user_review

['No Review',
 'No Review',
 'No Review',
 'No Review',
 'Coffee forward beer that is good but not great. And I’m used to greatness from founders when it comes to their stouts.',
 'No Review',
 "poured from the tap of my kegerator into a porter/stout glass  A - 2 fingers medium/dark tan over a body of pitch black; laces the glass in nice rings  S - lots of chocolate, coffee, roasted malts, suggestion of dark fruits  T - chocolate, rich black coffee, hint of dark fruitiness  M - thick full body, medium carbonation  O - I'm glad I bought this by the keg!\xa0",
 'No Review',
 'No Review',
 'Look Founders sure craft some of the best "looking" delicacies in the micro-market, and this one left no room for dissapointment. One of the most opaque stouts I have seen, with a rich burgundy red head to boot, which had dissipated within minutes, leaving behind a ebony jewel. A truly bold example, and a textbook stout aesthectic.  Smell Chocolate! Creamy, succulent, mouth-watering cocoa hit me immedi

In [264]:
df_reviews = pd.DataFrame({
    'url': beer_url,
    'user_score': user_score,
    'score_breakdown' : score_breakdown,
    'user_name': user_names,
    'review': user_review
})

df_reviews.head()

,review,score_breakdown,url,user_name,user_score
0,No Review,look: 4.25 | smell: 4.75 | taste: 5 | feel: 4....,/beer/profile/1199/11757/,Nobel96,4.82
1,No Review,look: 5 | smell: 4.75 | taste: 4.75 | feel: 5 ...,/beer/profile/1199/11757/,Batman11,4.79
2,No Review,look: 4 | smell: 4 | taste: 4 | feel: 4 | ove...,/beer/profile/1199/11757/,Bloodsnot77,4
3,No Review,look: 4 | smell: 4.25 | taste: 4 | feel: 4.25 ...,/beer/profile/1199/11757/,jrc1093,4.14
4,Coffee forward beer that is good but not great...,look: 4.25 | smell: 4 | taste: 4 | feel: 4.25 ...,/beer/profile/1199/11757/,Maudrey,4.09


In [272]:
# df_reviews.to_csv('./df_reviews_testing.csv', index=False)

In [276]:
# pd.read_csv('./df_reviews_testing.csv', encoding = "latin1")

,review,score_breakdown,url,user_name,user_score
0,No Review,look: 4.25 | smell: 4.75 | taste: 5 | feel: 4....,/beer/profile/1199/11757/,Nobel96,4.82
1,No Review,look: 5 | smell: 4.75 | taste: 4.75 | feel: 5 ...,/beer/profile/1199/11757/,Batman11,4.79
2,No Review,look: 4 | smell: 4 | taste: 4 | feel: 4 | ove...,/beer/profile/1199/11757/,Bloodsnot77,4.00
3,No Review,look: 4 | smell: 4.25 | taste: 4 | feel: 4.25 ...,/beer/profile/1199/11757/,jrc1093,4.14
4,Coffee forward beer that is good but not great...,look: 4.25 | smell: 4 | taste: 4 | feel: 4.25 ...,/beer/profile/1199/11757/,Maudrey,4.09
5,No Review,look: 4.5 | smell: 4.5 | taste: 3.5 | feel: 3....,/beer/profile/1199/11757/,ejmontoy1,3.78
6,poured from the tap of my kegerator into a por...,look: 4.5 | smell: 4 | taste: 4 | feel: 4 | o...,/beer/profile/1199/11757/,VforVinny,4.08
7,No Review,look: 4.75 | smell: 4.25 | taste: 4.5 | feel: ...,/beer/profile/1199/11757/,lwillitz,4.41
8,No Review,look: 4 | smell: 4 | taste: 4 | feel: 4 | ove...,/beer/profile/1199/11757/,Bdasilva1077,4.00
9,"Look Founders sure craft some of the best ""loo...",look: 4.5 | smell: 4.5 | taste: 4.75 | feel: 4...,/beer/profile/1199/11757/,spongebob2,4.60


### Writing Data to a csv

In [309]:
# for url in df_beer['url'][:1]:
def get_user_reviews_csv(url):
    counter = 0
    review_number = 1
    for i in range(0,2):
        driver.get(f'https://www.beeradvocate.com{url}?view=beer&sort=&start={i*25}')
        sleep(2)
        beer_page = driver.page_source
        beer_page_soup = BeautifulSoup(beer_page, 'lxml')
        
        reviews = beer_page_soup.find_all('div', {'id':'rating_fullview_content_2'})
        
        counter += 1
        print(f'{url} -- page {counter}')
        
        
#         saving to a file
        for count, review in enumerate(reviews):
            score = review.find('span', {'class': 'BAscore_norm'}).text
            breakdown = review.find('span', {'class': 'muted'}).text
            u_names = review.find('a', {'class':'username'}).text
            try:
                r_text = review_cleaner(reviews[count].text)
            except:
                r_text = "No Review"
                
            master_list = [str(review_number), url, score, breakdown, u_names, r_text]
            with open('./user_reviews.csv', 'a+') as f:
                print(','.join(master_list), file=f)
            
            review_number += 1
    
    sleep(2)

In [310]:
df_beer['url'][0:1].apply(get_user_reviews_csv)

/beer/profile/1199/11757/ -- page 1
/beer/profile/1199/11757/ -- page 2


1    None
Name: url, dtype: object